# Reaction Diffusion Systems

The Gray-Scott model of a reaction diffusion system with two interacting species U and V is given by:

$$
\begin{aligned}
\frac{\partial{u}}{\partial{t}} &= D_u\nabla^2u - uv^2 + f(1-u) \\
\frac{\partial{v}}{\partial{t}} &= D_v\nabla^2v + uv^2 - (f+k)v
\end{aligned}
$$

where $u$ and $v$ represent the concentrations of the two species $U$ and $V$, respectively, $D_u$ and $D_v$ their respective diffusion constants, and $f$ and $k$ the feed rate and kill rate, respectively. $\nabla^2$ is the Laplace operator, which in this two-dimensional problem is given by:

In [ ]:
Laplacian[u[x, y], {x, y}]

For different parameter choices, this model can producing very diverse patterns which mimic patterns found in nature, such as spots and stripes.  

At the microscopic level, two chemical reactions govern the system above:

$$
\begin{aligned}
U + 2V &\rightarrow 3V \\
V &\rightarrow P
\end{aligned}
$$

where $P$ represents an inert product which depletes the amount of $V$ present. Our goal will be to explore the variety of interesting visual patterns this simple yet rich interacting system can exhibit. For reference, the following website offers an interactive look into the phase diagram spanned by the $f$ and $k$ parameters: [Reaction-Diffusion by the Gray-Scott Model: Pearson's Parametrization](http://mrob.com/pub/comp/xmorphia/index.html).

## Finite Element Method

We'll implement this using the finite element method.  
This will give us flexibility over the domain and boundary conditions to solver over.

In [ ]:
eqn["gray-scott"][Du_, Dv_, f_, k_] = {
    Derivative[1, 0, 0][u][t, x, y] + Inactive[Div][(-Du Inactive[Grad][u[t, x, y], {x, y}]), {x,y}] +
       (v[t, x, y]^2 + f) u[t, x, y] == f, 
    Derivative[1, 0, 0][u][v, x, y] + Inactive[Div][(-Dv Inactive[Grad][v[t, x, y], {x, y}]), {x,y}] +
        (-u[t, x, y] v[t, x, y] + f + k) v[t, x, y] == 0
    };

We need some initial conditions.  
E.g. a circular seed of V in a sea of constant U

In [ ]:
ics["single-seed"] = {
   u[0, x, y] == 1/2,
   v[0, x, y] == If[x^2 + y^2 <= 1/40, 1, 0]
   };

We can solve this using fixed (Dirichlet) boundary conditions:

In [ ]:
bcs["fixed"] = DirichletCondition[{u[t, x, y] == 0, v[t, x, y] == 0}, True];

Over a disk domain:

In [ ]:
region["disk"] = Disk[];

And solve the pde using the FEM (note: this takes a while):

In [ ]:
{ufun["gs", "single-seed", "disk"], 
   vfun["gs", "single-seed", "disk"]} =
  NDSolveValue[{eqn["gray-scott"][2 10^-5, 5 10^-6, 1/25, 3/50], 
    bcs["fixed"], ics["single-seed"]}, {u, v}, {x, y} \[Element] 
    region["disk"], {t, 0, 2500}, 
   Method -> {"PDEDiscretization" -> {"MethodOfLines", 
       "SpatialDiscretization" -> {"FiniteElement", 
         "MeshOptions" -> {"MaxCellMeasure" -> 0.0005}}}}];

We can visualize the evolution of species V:

In [ ]:
frames["gs", "single-seed", "disk"] = With[{
    vRange =MinMax[vfun["gs", "single-seed", "disk"]["ValuesOnGrid"]], 
    sol = vfun["gs", "single-seed", "disk"], reg = region["disk"]},
   Table[ContourPlot[sol[t, x, y], {x, y} \[Element] reg, 
     PlotRange -> All, Frame -> None, Axes -> None, ColorFunction -> Hue, 
     Contours -> (Subdivide[#1, #2, 4] & @@ vRange), PlotPoints -> 50,MaxRecursion -> 3], 
     {t, Subdivide[90, 1930, 23]}]];
     
Rasterize[Multicolumn[frames["gs", "single-seed", "disk"], 8, Appearance -> "Horizontal"], ImageSize -> 800]

## Spectral Methods

The FEM is appealing because of its generality of domains and boundary conditions. However, the above runs quite slowly..  

Here, we implement an alternative spectral method, which will run significantly faster.  
**Note:** The code is adapted from initial code by Prof. Craig Carter.

First, we setup a finite grid of wave-vectors (dual-lattice vectors for the the domain we wish to solve)

In [ ]:
spectralWavevector[nSize_] := Module[{baseVec},
  baseVec = Range[-(1 + 1/nSize), 1, 2/nSize];  
  If[EvenQ[nSize],
   baseVec = Range[-1, 1/1, 2/nSize];
   baseVec = Rest[baseVec];
   baseVec = RotateLeft[baseVec, nSize/2 - 1], 
   
   baseVec = Range[-(1 + 1/nSize), 1 + 1/nSize, 2/nSize];
   baseVec = Most[Rest[baseVec]];
   baseVec = RotateLeft[baseVec, (nSize - 1)/2]
   ];
  N[Pi] baseVec
  ]

In [ ]:
spectralWavevectorSquared[{nx_Integer, ny_Integer}, preMultiply_ : True] := 
 Module[{kx, ky, kMagnitudeSquared},
  kx = If[preMultiply, nx, 1] Transpose[
     ConstantArray[spectralWavevector[nx], ny]]; 
  ky = If[preMultiply, ny, 1] ConstantArray[spectralWavevector[ny], nx]; 
  kMagnitudeSquared  = kx^2 + ky^2
  ]
  
ArrayPlot[ wavevectors["gs", "spectral"] = spectralWavevectorSquared[{256, 256}]]

Next, we formulate our pde in Fourier space:

In [ ]:
spectralGrayScott[{u_, v_}, {diffU_, diffV_, f_, k_},  dt_][kMagnitudeSquared_] :=
 Block[
  {fourierU = Fourier[u], 
   fourierV =  Fourier[v],
   uvSquared  = u v^2, 
   fourieruvSquared 
   },
  fourieruvSquared = Fourier[ uvSquared];
  {
   Re@InverseFourier[(fourierU +  
        dt (Fourier[f (1 - u)] - fourieruvSquared))/(1 + 
        diffU*dt*kMagnitudeSquared)],
   Re@InverseFourier[(fourierV + 
        dt (fourieruvSquared - Fourier[(f + k) v]))/(1 + 
        diffV*dt*kMagnitudeSquared)]
   }
  ]

We need some initial conditions to solve, Let's start with the simple single-seed as before:

In [ ]:
uVals["gs", "single-seed", "spectral"] = ConstantArray[0.5, {256, 256}];
vVals["gs", "single-seed", "spectral"] = GaussianFilter[DiskMatrix[16, {256, 256}], 4];

In [ ]:
storedResult["gs", "single-seed", "spectral"] =
  Reap[Do[
     {uVals["gs", "single-seed", "spectral"], 
       vVals["gs", "single-seed", "spectral"]} =
      spectralGrayScott[{uVals["gs", "single-seed", "spectral"], 
         vVals["gs", "single-seed", "spectral"]}, {2. 10^-5, 5. 10^-6,
          1/25., 3/50.}, 0.5][wavevectors["gs", "spectral"]];
     If[Mod[t, 165] == 0, 
      Sow[vVals["gs", "single-seed", "spectral"]]]
     , {t, 1, 4000}]][[2, 1]];

In [ ]:
Rasterize[Multicolumn[
  ArrayPlot[#, ImageSize -> 200, Frame -> False] & /@ storedResult["gs", "single-seed", "spectral"], 
    8,Appearance -> "Horizontal"], ImageSize -> 800]

Let's also start with some ink-blobs as initial conditions. We'll use our 'coarse spinodal' CA to generate these. We'll also change our f and k parameters to see 'dots and stripes'

In [ ]:
randomInkBlob[{nx_, ny_}, iterations_ : 100] :=
 Module[{rule = {976, {2, 1}, {1, 1}}, init = SparseArray[RandomInteger[{0, 1}, {nx, ny}]]},
  GaussianFilter[First@CellularAutomaton[rule, init, {{iterations}}], nx/64]]

In [ ]:
ArrayPlot /@ Table[randomInkBlob[{256, 256}], 2]

In [ ]:
uVals["gs", "ink-blob", "spectral"] = randomInkBlob[{256, 256}];
vVals["gs", "ink-blob", "spectral"] = randomInkBlob[{256, 256}];

In [ ]:
storedResult["gs", "ink-blob", "spectral"] =
  Reap[Do[
     {uVals["gs", "ink-blob", "spectral"], 
       vVals["gs", "ink-blob", "spectral"]} =
      spectralGrayScott[{uVals["gs", "ink-blob", "spectral"], 
         vVals["gs", "ink-blob", "spectral"]}, {2. 10^-5, 5. 10^-6, 
         0.024, 0.056}, 0.125][wavevectors["gs", "spectral"]];
     If[Mod[t, 165] == 0, Sow[vVals["gs", "ink-blob", "spectral"]]]
     , {t, 1, 4000}]][[2, 1]];

In [ ]:
Rasterize[Multicolumn[
  ArrayPlot[#, ImageSize -> 200, Frame -> False] & /@ storedResult["gs", "ink-blob", "spectral"], 
  8, Appearance -> "Horizontal"], ImageSize -> 800]

## Cahn Hilliard Equation for Spinodal Decomposition

Another important diffusion system in materials science is the Cahn Hilliard equation, which describes phase-separation in two-phase systems:

$$
\frac{\partial c}{\partial t} = M_0 \left[ \frac{\partial^2 f^{\mathrm{hom}}}{\partial c^2} \nabla^2 c - 2K \nabla^4 c \right]
$$

The first term on the right is diffusive, the second term accounts for interfacial energy penalties arising from concentration gradients.   
We'll use it to model spinodal decomposition.

First, we need a particular free energy potential f^hom for how the two phases interact. We'll choose a simple function:

In [ ]:
freeEnergy[phi_] = phi^2 (1 - phi)^2;
Plot[freeEnergy[phi], {phi, -1/2, 3/2}]

We'll need the gradient df/dc:

In [ ]:
Simplify[freeEnergy'[phi]]

In [ ]:
freeEnergyDerivative[phi_] = phi (2 + phi (-6 + 4 phi));

We'll re-use our spectral wave-vectors functions from above.
Finally, we implement our 4th-order pde:

In [ ]:
spectralCahnHilliard[u_, kappa_,  dt_][kMagnitudeSquared_,kMagnitudeFourth_] :=
 Block[{
   fourierDf = Fourier[freeEnergyDerivative[u]],
   fourierU = Fourier[u] ,
   part1,
   part2
   },
  part1 = dt kMagnitudeSquared fourierDf;
  part2 = (fourierU - part1)/(1 + dt kappa kMagnitudeFourth);
  Re[InverseFourier[part2]]]

We'll use Obama's image again.  
We rescale the values to initially lie very close to the unstable 'bump' in our free energy potential.
And evolve down to the two phase-separation minima (0 and 1)

In [ ]:
obamaImage = ColorConvert[ImageResize[Import["https://www.beyonddream.com/images/product/23892024.jpg"],500], "Grayscale"];
obamaData = ImageData[obamaImage];
obamaVals["ch", "obama"] = Rescale[obamaData, {0, 1}, {0.4999, 0.5001}];

In [ ]:
wavevectorsSquared["ch", "obama"] = spectralWavevectorSquared[Dimensions[obamaVals["ch", "obama"]],  False];
wavevectorsFourth["ch", "obama"] = wavevectorsSquared["ch", "obama"]^2;

In [ ]:
storedResult["ch", "obama"] =
  Reap[Do[
     obamaVals["ch", "obama"] =
      spectralCahnHilliard[obamaVals["ch", "obama"], 7.5, 1][
       wavevectorsSquared["ch", "obama"], 
       wavevectorsFourth["ch", "obama"]
       ];
     If[Mod[t, 40] == 0, Sow[obamaVals["ch", "obama"]]]
     , {t, 1, 1000}]][[2, 1]];

Finally, we visualize the decomposition:

In [ ]:
adjustRange[0] = obamaImage;
adjustRange[frameNumber_] := storedResult["ch", "obama"][[frameNumber]] // Image // ImageAdjust

In [ ]:
Rasterize[Multicolumn[adjustRange /@ Range[0, 23], 8, Appearance -> "Horizontal"], ImageSize -> 800]